In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [1]:
from IPython.display import Image
%matplotlib inline

In [3]:
import torch
import torch.nn as nn
torch.manual_seed(1)
rnn_layer= nn.RNN(input_size=5, hidden_size=2, num_layers=1, batch_first=True)

In [6]:
w_xh = rnn_layer.weight_ih_10
w_hh = rnn_layer.weight_hh_10
b_xh = rnn_layer.bias_ih_10
b_hh = rnn_layer.bias_ih_10


AttributeError: 'RNN' object has no attribute 'weight_ih_10'

In [7]:

w_xh = rnn_layer.weight_ih_l0
w_hh = rnn_layer.weight_hh_l0
b_xh = rnn_layer.bias_ih_l0
b_hh = rnn_layer.bias_hh_l0

In [8]:
w_xh

Parameter containing:
tensor([[ 0.3643, -0.3121, -0.1371,  0.3319, -0.6657],
        [ 0.4241, -0.1455,  0.3597,  0.0983, -0.0866]], requires_grad=True)

In [9]:
print("W_xh 크기" , w_xh.shape)
print("W_hh 크기" , w_hh.shape)
print("b_xh 크기" , b_xh.shape)
print("b_hh 크기" , b_hh.shape)

W_xh 크기 torch.Size([2, 5])
W_hh 크기 torch.Size([2, 2])
b_xh 크기 torch.Size([2])
b_hh 크기 torch.Size([2])


In [14]:
x_seq = torch.tensor([[1.0]*5,[2.0]*5,[3.0]*5]).float()

In [15]:
x_seq.reshape(-1,3,5).shape

torch.Size([1, 3, 5])

In [16]:
x_seq.ndim

2

In [17]:
output, hn = rnn_layer(x_seq.reshape(-1 ,3,5))

In [24]:
out_manual = []
for t in range(3):
    xt = torch.reshape(x_seq[t], (1,5))
    print(f"타임 스텝 {t} --> ")
    print("입력    : ", xt.numpy())
   
    ht = torch.matmul(xt, torch.transpose(w_xh, 0, 1)) + b_xh
    print(" 은닉  : ", ht.detach().numpy())


    prev_h = torch.zeros((ht.shape))
    ot = ht + torch.matmul(prev_h, torch.transpose(w_hh, 0, 1)) + b_hh
    ot = torch.tanh(ot)
    break


타임 스텝 0 --> 
입력    :  [[1. 1. 1. 1. 1.]]
 은닉  :  [[-0.4701929   0.58639044]]


In [25]:
output

tensor([[[-0.3520,  0.5253],
         [-0.6842,  0.7607],
         [-0.8649,  0.9047]]], grad_fn=<TransposeBackward1>)

In [26]:
out_manual = []
for t in range(3):
    xt = torch.reshape(x_seq[t], (1,5))
    print(f"타임 스텝 {t} --> ")
    print("입력    : ", xt.numpy())
   
    ht = torch.matmul(xt, torch.transpose(w_xh, 0, 1)) + b_xh
    print(" 은닉  : ", ht.detach().numpy())


    if t>0:
        prev_h = out_manual[t-1]
    else:
        prev_h = torch.zeros((ht.shape))


    ot = ht + torch.matmul(prev_h, torch.transpose(w_hh, 0, 1)) + b_hh
    ot = torch.tanh(ot)
    out_manual.append(ot)
    print ("  출력(수동)   : ", ot)
    print ("  RNN 모듈에서 계산된 값 : ", output[:, t].detach().numpy())

타임 스텝 0 --> 
입력    :  [[1. 1. 1. 1. 1.]]
 은닉  :  [[-0.4701929   0.58639044]]
  출력(수동)   :  tensor([[-0.3520,  0.5253]], grad_fn=<TanhBackward0>)
  RNN 모듈에서 계산된 값 :  [[-0.3519801   0.52525216]]
타임 스텝 1 --> 
입력    :  [[2. 2. 2. 2. 2.]]
 은닉  :  [[-0.88883156  1.2364398 ]]
  출력(수동)   :  tensor([[-0.6842,  0.7607]], grad_fn=<TanhBackward0>)
  RNN 모듈에서 계산된 값 :  [[-0.68424344  0.76074266]]
타임 스텝 2 --> 
입력    :  [[3. 3. 3. 3. 3.]]
 은닉  :  [[-1.3074702  1.8864892]]
  출력(수동)   :  tensor([[-0.8649,  0.9047]], grad_fn=<TanhBackward0>)
  RNN 모듈에서 계산된 값 :  [[-0.8649416  0.9046636]]
